We start by importing th Spacetime Variables module, which contains all the functionality we will need to generate our Z4c equations.

In [1]:
from SpacetimeVar import *

We set a path for our source files to be generated.

In [2]:
DiffOrder = 4
KOsigma = 0.1
DiffOrderUpDn = 4

In [3]:
if DiffOrderUpDn%2 == 0:
    strUpCn = 'up'
else:
    strUpCn = 'cn'

In [4]:
dim = 3
path = "../Source/Z4c/"

Let us begin by writing the beginnings of our RHS file that will contain the right hand sides of our evolutions equations. These statements set up the grid to which our multifab grid functions will live.

In [5]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")
fileRHS.write("#ifndef ET_INTEGRATION_RHS_K_H \n")
fileRHS.write("#define ET_INTEGRATION_RHS_K_H \n\n")

fileRHS.write("#include <AMReX_REAL.H> \n")
fileRHS.write("#include <AMReX_Array4.H> \n")
fileRHS.write("#include <ET_Integration_Setup.H> \n\n")

fileRHS.write("AMREX_GPU_DEVICE \ninline \nvoid \n")
fileRHS.write("state_rhs(int i, int j, int k, \n")
fileRHS.write("        amrex::Array4<amrex::Real> const& rhs_fab, \n")
fileRHS.write("        amrex::Array4<amrex::Real const> const& state_fab, \n")
fileRHS.write("        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, \n")
fileRHS.write("        const amrex::GeometryData& geom) noexcept \n{\n")
fileRHS.write("        const auto domain_xlo = geom.ProbLo(); \n\n")
fileRHS.write("        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; \n")
fileRHS.write("        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; \n")
fileRHS.write("        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2]; \n\n")

71

Now we will declare our evolution variables including the gauge variables alpha and beta (and B optionally).  

In [6]:
stVar.decl = []  #We want a list of declared variables that we will cycle through when we convert our RHS equations.

chi = stVar('chi', declare = True) #Scalar variables like chi can be declared with stVar.
gamtildeLL = stVarRank2('gamtildeLL', sym = 'sym01', declare = True)  #gamtildeLL is rank 2, so it is declared with stVarRank2
Khat = stVar('Khat', declare = True)
AtildeLL = stVarRank2('AtildeLL', sym = 'sym01', declare = True)
theta = stVar('theta', declare = True)
GamtildeU = stVarRank1('GamtildeU', declare = True)

alpha = stVar('alpha', declare = True)
betaU = stVarRank1('betaU', declare = True)

We are now ready to declare our evolution variables in the RHS. We do this with AMReXDeclare(), meaning that these variables are defined by their associated grid functions.  Below we will define other intermediate variables (like the Ricci curvature etc. in terms of other (declared and not declared) variables. 

In [7]:
fileRHS.write(chi.AMReXDeclare())
fileRHS.write(gamtildeLL.AMReXDeclare())
fileRHS.write(Khat.AMReXDeclare())
fileRHS.write(AtildeLL.AMReXDeclare())
fileRHS.write(theta.AMReXDeclare())
fileRHS.write(GamtildeU.AMReXDeclare())

fileRHS.write(alpha.AMReXDeclare())
fileRHS.write(betaU.AMReXDeclare())

186

In [8]:
betaUp = stVarRank1('betaUp')
betaDn = stVarRank1('betaDn')

In [9]:
betaString = """
        amrex::Real betaUp0 = 0;
        amrex::Real betaDn0 = 0;
        if(betaU0 > 0)
        {
            betaUp0 = 1;
            betaDn0 = 0;
        }
        else if(betaU0 < 0)
        {
            betaUp0 = 0;
            betaDn0 = 1;
        }
        
        amrex::Real betaUp1 = 0;
        amrex::Real betaDn1 = 0;
        if(betaU1 > 0)
        {
            betaUp1 = 1;
            betaDn1 = 0;
        }
        else if(betaU1 < 0)
        {
            betaUp1 = 0;
            betaDn1 = 1;
        }
        
        amrex::Real betaUp2 = 0;
        amrex::Real betaDn2 = 0;
        if(betaU2 > 0)
        {
            betaUp2 = 1;
            betaDn2 = 0;
        }
        else if(betaU2 < 0)
        {
            betaUp2 = 0;
            betaDn2 = 1;
        }


"""
fileRHS.write(betaString)

792

In [10]:
betaDchi = stVar('betaDchi')
Dupchi = stVarRank1('Dupchi')
Dupchi.var = DstVar(chi, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).var

Ddnchi = stVarRank1('Ddnphi')
Ddnchi.var = DstVar(chi, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).var

for k in range(3):
    betaDchi.var += betaU.symb[k]*(betaUp.symb[k]*Dupchi.symb[k]+betaDn.symb[k]*Ddnchi.symb[k])
    
betaDgamtildeLL = stVarRank2('betaDgamtildeLL')
DupgamtildeLL = stVarRank3('DupgamtildeLL')
DupgamtildeLL.var = DstVarRank2(gamtildeLL,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).var

DdngamtildeLL = stVarRank3('DdngamtildeLL')
DdngamtildeLL.var = DstVarRank2(gamtildeLL,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).var

for i in range(3):
    for j in range(3):
        for k in range(3):
            betaDgamtildeLL.var[i][j] += betaU.symb[k]*(betaUp.symb[k]*DupgamtildeLL.symb[i][j][k]+betaDn.symb[k]*DdngamtildeLL.symb[i][j][k])

betaDKhat = stVar('betaDKhat')
DupKhat = stVarRank1('DupKhat')
DupKhat.var = DstVar(Khat, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).var

DdnKhat = stVarRank1('DdnKhat')
DdnKhat.var = DstVar(Khat, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).var

for k in range(3):
    betaDKhat.var += betaU.symb[k]*(betaUp.symb[k]*DupKhat.symb[k]+betaDn.symb[k]*DdnKhat.symb[k])
    
betaDAtildeLL = stVarRank2('betaDAtildeLL')
DupAtildeLL = stVarRank3('DupAtildeLL')
DupAtildeLL.var = DstVarRank2(AtildeLL,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).var

DdnAtildeLL = stVarRank3('DdnAtildeLL')
DdnAtildeLL.var = DstVarRank2(AtildeLL,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).var

for i in range(3):
    for j in range(3):
        for k in range(3):
            betaDAtildeLL.var[i][j] += betaU.symb[k]*(betaUp.symb[k]*DupAtildeLL.symb[i][j][k]+betaDn.symb[k]*DdnAtildeLL.symb[i][j][k])

betaDtheta = stVar('betaDtheta')
Duptheta = stVarRank1('Duptheta')
Duptheta.var = DstVar(theta, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).var

Ddntheta = stVarRank1('Ddntheta')
Ddntheta.var = DstVar(theta, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).var

for k in range(3):
    betaDtheta.var += betaU.symb[k]*(betaUp.symb[k]*Duptheta.symb[k]+betaDn.symb[k]*Ddntheta.symb[k])
    
betaDGamtildeU = stVarRank1('betaDGamtildeU')
DupGamtildeU = stVarRank2('DupGamtildeU')
DupGamtildeU.var = DstVarRank1(GamtildeU,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).var

DdnGamtildeU = stVarRank2('DdnGamtildeU')
DdnGamtildeU.var = DstVarRank1(GamtildeU,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).var

for i in range(3):
    for k in range(3):
        betaDGamtildeU.var[i] += betaU.symb[k]*(betaUp.symb[k]*DupGamtildeU.symb[i][k]+betaDn.symb[k]*DdnGamtildeU.symb[i][k])

betaDalpha = stVar('betaDalpha')
Dupalpha = stVarRank1('Dupalpha')
Dupalpha.var = DstVar(alpha, difftype = 1,CnUpDnRank1=strUpCn,orderD = DiffOrderUpDn).var

Ddnalpha = stVarRank1('Ddnalpha')
Ddnalpha.var = DstVar(alpha, difftype = 1,CnUpDnRank1='dn',orderD = DiffOrderUpDn).var

for k in range(3):
    betaDalpha.var += betaU.symb[k]*(betaUp.symb[k]*Dupalpha.symb[k]+betaDn.symb[k]*Ddnalpha.symb[k])
    
betaDbetaU = stVarRank1('betaDbetaU')
DupbetaU = stVarRank2('DupbetaU')
DupbetaU.var = DstVarRank1(betaU,difftype = 1, CnUpDnRank1=strUpCn,orderD=DiffOrderUpDn).var

DdnbetaU = stVarRank2('DdnbetaU')
DdnbetaU.var = DstVarRank1(betaU,difftype = 1, CnUpDnRank1='dn',orderD=DiffOrderUpDn).var

for i in range(3):
    for k in range(3):
        betaDbetaU.var[i] += betaU.symb[k]*(betaUp.symb[k]*DupbetaU.symb[i][k]+betaDn.symb[k]*DdnbetaU.symb[i][k])


In [11]:
fileRHS.write(Dupchi.AMReXDeclare())
fileRHS.write(Ddnchi.AMReXDeclare())
fileRHS.write(betaDchi.AMReXReal())

fileRHS.write(DupgamtildeLL.AMReXDeclare())
fileRHS.write(DdngamtildeLL.AMReXDeclare())
fileRHS.write(betaDgamtildeLL.AMReXReal())

fileRHS.write(DupKhat.AMReXDeclare())
fileRHS.write(DdnKhat.AMReXDeclare())
fileRHS.write(betaDKhat.AMReXReal())

fileRHS.write(DupAtildeLL.AMReXDeclare())
fileRHS.write(DdnAtildeLL.AMReXDeclare())
fileRHS.write(betaDAtildeLL.AMReXReal())

fileRHS.write(Duptheta.AMReXDeclare())
fileRHS.write(Ddntheta.AMReXDeclare())
fileRHS.write(betaDtheta.AMReXReal())

fileRHS.write(DupGamtildeU.AMReXDeclare())
fileRHS.write(DdnGamtildeU.AMReXDeclare())
fileRHS.write(betaDGamtildeU.AMReXReal())

fileRHS.write(Dupalpha.AMReXDeclare())
fileRHS.write(Ddnalpha.AMReXDeclare())
fileRHS.write(betaDalpha.AMReXReal())

fileRHS.write(DupbetaU.AMReXDeclare())
fileRHS.write(DdnbetaU.AMReXDeclare())
fileRHS.write(betaDbetaU.AMReXReal())

558

In [12]:
DiffOrder = 2
KOsigma = 0.1

In [13]:
fileRHS.write(DstVar(chi, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(chi, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(chi, 'KO', orderD = DiffOrder, sigma = KOsigma).AMReXDeclare())

fileRHS.write(DstVarRank2(gamtildeLL,1,orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank2(gamtildeLL,2,orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank2(gamtildeLL, 'KO', sigma = KOsigma).AMReXDeclare())

fileRHS.write(DstVar(Khat,1,orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(Khat, 'KO', sigma = KOsigma,orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank2(AtildeLL,1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank2(AtildeLL, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVar(theta,1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(theta, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank1(GamtildeU, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(GamtildeU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVar(alpha, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(alpha, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(alpha, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

fileRHS.write(DstVarRank1(betaU, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(betaU, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVarRank1(betaU, 'KO', sigma = KOsigma, orderD = DiffOrder).AMReXDeclare())

2100

In [14]:
gamtildeUU = stVarRank2('gamtildeUU')
for i in range(dim):
    for j in range(dim):
         gamtildeUU.var[i][j] += sp.simplify(sp.Matrix(gamtildeLL.symb).inv()[i,j])

In [15]:
detgamtilde = stVar('detgamtilde')
detgamtilde.var = sp.simplify(sp.Matrix(gamtildeLL.symb).det())
print(detgamtilde.var)

gamtildeLL00*gamtildeLL11*gamtildeLL22 - gamtildeLL00*gamtildeLL12*gamtildeLL21 - gamtildeLL01*gamtildeLL10*gamtildeLL22 + gamtildeLL01*gamtildeLL12*gamtildeLL20 + gamtildeLL02*gamtildeLL10*gamtildeLL21 - gamtildeLL02*gamtildeLL11*gamtildeLL20


In [16]:
fileRHS.write(gamtildeUU.AMReXReal())

3026

In [17]:
DgamtildeLLL = stVarRank3('DgambarLLL')
GamtildeULL = stVarRank3('GamtildeULL')
GamtildeLLL = stVarRank3('GamtildeLLL')
GamULL = stVarRank3('GamULL')

for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            DgamtildeLLL.var[i][j][k] = Dsymb(gamtildeLL.symb[i][j],k)
            
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                GamtildeULL.var[i][j][k] += 1/2*gamtildeUU.symb[i][l]*(DgamtildeLLL.var[l][j][k]+DgamtildeLLL.var[l][k][j]-DgamtildeLLL.var[j][k][l])
                
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            for l in range(dim):
                GamtildeLLL.var[i][j][k] += gamtildeLL.symb[i][l]*GamtildeULL.var[l][j][k]
                
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            GamULL.var[i][j][k] += GamtildeULL.symb[i][j][k] - 1/2*((sp.eye(3)[i,j]/chi.symb)*Dsymb(chi.symb,k)+(sp.eye(3)[i,k]/chi.symb)*Dsymb(chi.symb,j))
            for l in range(3):
                GamULL.var[i][j][k] += 1/2*gamtildeLL.symb[j][k]*gamtildeUU.symb[i][l]*Dsymb(chi.symb,l)/chi.symb
                               
                

In [18]:
fileRHS.write(GamtildeULL.AMReXReal())
fileRHS.write(GamtildeLLL.AMReXReal())
fileRHS.write(GamULL.AMReXReal())

4938

In [19]:
DDalphaLL = stVarRank2('DDalphaLL')
for i in range(3):
    for j in range(3):
        DDalphaLL.var[i][j] += Dsymb(alpha.symb,str(i)+str(j))
        for k in range(3):
            DDalphaLL.var[i][j] += -GamULL.symb[k][i][j]*Dsymb(alpha.symb,str(k))

In [20]:
fileRHS.write(DDalphaLL.AMReXReal())

954

In [21]:
gamLL = stVarRank2('gamLL')
for i in range(3):
    for j in range(3):
        gamLL.var[i][j] = (chi.symb**(-1))*gamtildeLL.symb[i][j]
        
gamUU = stVarRank2('gamUU')
for i in range(3):
    for j in range(3):
        gamUU.var[i][j] = chi.symb*gamtildeUU.symb[i][j]

In [22]:
fileRHS.write(gamLL.AMReXReal())
fileRHS.write(gamUU.AMReXReal())

432

In [23]:
AtildeUU = stVarRank2('AtildeUU')
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                AtildeUU.var[i][j] += gamtildeUU.symb[i][k]*gamtildeUU.symb[j][l]*AtildeLL.symb[k][l]
                
AtildeUL = stVarRank2('AtildeUL')
for i in range(3):
    for j in range(3):
        for k in range(3):
            AtildeUL.var[i][j] += gamtildeUU.symb[i][k]*AtildeLL.symb[k][j]

In [24]:
TrAtilde = stVar('TrAtilde')
for i in range(3):
    for j in range(3):
        TrAtilde.var += gamtildeUU.symb[i][j]*AtildeLL.symb[i][j]
        
print(TrAtilde.var)

0 + AtildeLL00*gamtildeUU00 + AtildeLL01*gamtildeUU01 + AtildeLL02*gamtildeUU02 + AtildeLL10*gamtildeUU10 + AtildeLL11*gamtildeUU11 + AtildeLL12*gamtildeUU12 + AtildeLL20*gamtildeUU20 + AtildeLL21*gamtildeUU21 + AtildeLL22*gamtildeUU22


In [25]:
fileRHS.write(AtildeUU.AMReXReal())
fileRHS.write(AtildeUL.AMReXReal())

990

In [26]:
DDalphaTFLL=stVarRank2('DDalphaTFLL')

for i in range(3):
    for j in range(3):
        DDalphaTFLL.var[i][j] += DDalphaLL.symb[i][j]
        for k in range(3):
            for l in range(3):
                DDalphaTFLL.var[i][j] += -1/3*gamLL.symb[i][j]*(gamUU.symb[k][l]*DDalphaLL.symb[k][l])

In [27]:
fileRHS.write(DDalphaTFLL.AMReXReal())

4500

In [28]:
GamtildeDU = stVarRank1('GamtildeDU')
for i in range(dim):
    for j in range(dim):
        for k in range(dim):
            GamtildeDU.var[i] += GamtildeULL.symb[i][j][k]*gamtildeUU.symb[j][k] 

In [29]:
fileRHS.write(GamtildeDU.AMReXReal())

909

In [30]:
RtildeLL = stVarRank2('RtildeLL')

for i in range(3):
    for j in range(3):
        for l in range(3):
            for m in range(3):
                RtildeLL.var[i][j] += -1/2*gamtildeUU.symb[l][m]*Dsymb(gamtildeLL.symb[i][j],str(l)+str(m))
        for k in range(3):
            RtildeLL.var[i][j] += 1/2*(gamtildeLL.symb[k][i]*Dsymb(GamtildeU.symb[k],str(j))+gamtildeLL.symb[k][j]*Dsymb(GamtildeU.symb[k],str(i)))
            RtildeLL.var[i][j] += 1/2*(GamtildeDU.symb[k]*GamtildeLLL.symb[i][j][k]+GamtildeDU.symb[k]*GamtildeLLL.symb[j][i][k])
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):
                    RtildeLL.var[i][j] += gamtildeUU.symb[l][m]*(GamtildeULL.symb[k][l][i]*GamtildeLLL.symb[j][k][m]+GamtildeULL.symb[k][l][j]*GamtildeLLL.symb[i][k][m])
                    RtildeLL.var[i][j] += gamtildeUU.symb[l][m]*GamtildeULL.symb[k][i][m]*GamtildeLLL.symb[k][l][j]

In [31]:
fileRHS.write(RtildeLL.AMReXReal())

33534

In [32]:
DtildeDtildechiLL = stVarRank2('DtildeDtildechiLL')
for i in range(3):
    for j in range(3):
        DtildeDtildechiLL.var[i][j] += Dsymb(chi.symb,str(i)+str(j))
        for k in range(3):
            DtildeDtildechiLL.var[i][j] += -GamtildeULL.symb[k][i][j]*Dsymb(chi.symb,k)
            
TrDtildeDtildechi = stVar('TrDtildeDtildechi')
for i in range(3):
    for j in range(3):
        TrDtildeDtildechi.var += gamtildeUU.symb[i][j]*DtildeDtildechiLL.symb[i][j]

In [33]:
fileRHS.write(DtildeDtildechiLL.AMReXReal())
fileRHS.write(TrDtildeDtildechi.AMReXReal())

358

In [34]:
RchiLL = stVarRank2('RchiLL')            
for i in range(3):
    for j in range(3):
        RchiLL.var[i][j] += 1/2*(chi.symb**(-1))*(DtildeDtildechiLL.symb[i][j]+gamtildeLL.symb[i][j]*TrDtildeDtildechi.symb)
        RchiLL.var[i][j] += -1/4*(chi.symb**(-2))*Dsymb(chi.symb,str(i))*Dsymb(chi.symb,str(j))
        for l in range(3):
            RchiLL.var[i][j] += -3/4*(chi.symb**(-2))*gamtildeLL.symb[i][j]*Dsymb(chi.symb,str(l))*Dsymb(chi.symb,str(l))
        

In [35]:
fileRHS.write(RchiLL.AMReXReal())

2712

In [36]:
RLL = stVarRank2('RLL')
for i in range(3):
    for j in range(3):
        RLL.var[i][j] += RtildeLL.symb[i][j]+RchiLL.symb[i][j]

In [37]:
fileRHS.write(RLL.AMReXReal())

459

In [38]:
Rsclr = stVar('Rsclr')
for i in range(3):
    for j in range(3):
        Rsclr.var += gamUU.symb[i][j]*RLL.symb[i][j] 

In [39]:
fileRHS.write(Rsclr.AMReXReal())

175

In [40]:
RTFLL = stVarRank2('RTFLL')
for i in range(3):
    for j in range(3):
        RTFLL.var[i][j] = RLL.symb[i][j]-1/3*gamLL.symb[i][j]*Rsclr.symb

In [41]:
fileRHS.write(RTFLL.AMReXReal())

657

In [42]:
divTildebeta = stVar('divTildebeta')
for i in range(3):
    divTildebeta.var += Dsymb(betaU.symb[i],str(i))
    for k in range(3):
        divTildebeta.var += GamtildeULL.symb[i][i][k]*betaU.symb[k]

In [43]:
fileRHS.write(divTildebeta.AMReXReal())

287

In [44]:
RHS_chi = stVar('chi')
RHS_chi.var = 2/3*chi.symb*(alpha.symb*(Khat.symb+2*theta.symb)-divTildebeta.symb)
RHS_chi.var += betaDchi.symb 
    
RHS_chi.var += Dsymb(chi.symb,'KO')

In [45]:
fileRHS.write(RHS_chi.AMReXRHS())

122

In [46]:
RHS_gamtildeLL = stVarRank2('gamtildeLL',sym = 'sym01')

for i in range(3):
    for j in range(3):
        RHS_gamtildeLL.var[i][j] += -2*alpha.symb*AtildeLL.symb[i][j]
        RHS_gamtildeLL.var[i][j] += betaDgamtildeLL.symb[i][j]
        for k in range(3):
            RHS_gamtildeLL.var[i][j] += gamtildeLL.symb[k][i]*Dsymb(betaU.symb[k],str(j))+gamtildeLL.symb[k][j]*Dsymb(betaU.symb[k],str(i))
            RHS_gamtildeLL.var[i][j] += -2/3*gamtildeLL.symb[i][j]*Dsymb(betaU.symb[k],str(k))#+betaU.symb[k]*Dsymb(gamtildeLL.symb[i][j],str(k))
            
for i in range(3):
    for j in range(3):
        RHS_gamtildeLL.var[i][j] += Dsymb(gamtildeLL.symb[i][j],'KO')

In [47]:
fileRHS.write(RHS_gamtildeLL.AMReXRHS())

1935

In [48]:
kappa1 = stVar('kappa1')
kappa2 = stVar('kappa2')

kappa1.var = 0.02
kappa2.var = 0

RHS_Khat = stVar('Khat')
for i in range(3):
    for j in range(3):
        RHS_Khat.var += -gamUU.symb[i][j]*DDalphaLL.symb[j][i]+alpha.symb*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]

RHS_Khat.var += betaDKhat.symb
#for k in range(3):
 #   RHS_Khat.var += betaU.symb[k]*Dsymb(Khat.symb,str(k))

RHS_Khat.var += 1/3*alpha.symb*((Khat.symb+2*theta.symb)**2+kappa1.symb*(1-kappa2.symb)*theta.symb)
RHS_Khat.var += Dsymb(Khat.symb,'KO')

In [49]:
fileRHS.write(kappa1.AMReXReal())
fileRHS.write(kappa2.AMReXReal())
fileRHS.write(RHS_Khat.AMReXRHS())

617

In [50]:
RHS_AtildeLL = stVarRank2('AtildeLL', sym = 'sym01')
for i in range(3):
    for j in range(3):
        RHS_AtildeLL.var[i][j] += chi.symb*(-DDalphaTFLL.symb[i][j]+alpha.symb*RTFLL.symb[i][j])+alpha.symb*(Khat.symb+2*theta.symb)*AtildeLL.symb[i][j]
        RHS_AtildeLL.var[i][j] += betaDAtildeLL.symb[i][j]
        for k in range(3):
            RHS_AtildeLL.var[i][j] += -2*alpha.symb*AtildeLL.symb[i][k]*AtildeUL.symb[k][j]#+betaU.symb[k]*Dsymb(AtildeLL.symb[i][j],str(k))
            RHS_AtildeLL.var[i][j] += AtildeLL.symb[k][i]*Dsymb(betaU.symb[k],str(j))+AtildeLL.symb[k][j]*Dsymb(betaU.symb[k],str(i))-2/3*AtildeLL.symb[i][j]*Dsymb(betaU.symb[k],str(k))
            
for i in range(3):
    for j in range(3):
        RHS_AtildeLL.var[i][j] += Dsymb(AtildeLL.symb[i][j],'KO')

In [51]:
fileRHS.write(RHS_AtildeLL.AMReXRHS())

2718

In [52]:
RHS_theta = stVar('theta')
RHS_theta.var = 1/2*alpha.symb*(Rsclr.symb + 2/3*(Khat.symb+2*theta.symb)**2 - 2*kappa1.symb*(2+kappa2.symb)*theta.symb)
#for i in range(3):
 #   RHS_theta.var += betaU.symb[i]*Dsymb(theta.symb,str(i))

RHS_theta.var += betaDtheta.symb
for i in range(3):
    for j in range(3):
        RHS_theta.var += -1/2*alpha.symb*AtildeLL.symb[i][j]*AtildeUU.symb[i][j]

RHS_theta.var += Dsymb(theta.symb,'KO')

In [53]:
fileRHS.write(RHS_theta.AMReXRHS())

469

In [54]:
RHS_GamtildeU = stVarRank1('GamtildeU')

for i in range(3):
    RHS_GamtildeU.var[i] += betaDGamtildeU.symb[i]
    for j in range(3):
        RHS_GamtildeU.var[i] += -2*AtildeUU.symb[i][j]*Dsymb(alpha.symb,str(j))+2*alpha.symb*(-1/3*gamtildeUU.symb[i][j]*(2*Dsymb(Khat.symb,str(j))+Dsymb(theta.symb,str(j)))-3/2*AtildeUU.symb[i][j]*Dsymb(chi.symb,str(j))/chi.symb)
        RHS_GamtildeU.var[i] += -GamtildeDU.symb[j]*Dsymb(betaU.symb[i],str(j))+2/3*GamtildeDU.symb[i]*Dsymb(betaU.symb[j],str(j))#+betaU.symb[j]*Dsymb(GamtildeU.symb[i],str(j))
        for k in range(3):
            RHS_GamtildeU.var[i] += 2*alpha.symb*GamtildeULL.symb[i][j][k]*AtildeUU.symb[j][k]
            RHS_GamtildeU.var[i] += 1/3*gamtildeUU.symb[i][j]*Dsymb(betaU.symb[k],str(j)+str(k))+gamtildeUU.symb[j][k]*Dsymb(betaU.symb[i],str(j)+str(k))
            
for i in range(3):
    RHS_GamtildeU.var[i] += -2*alpha.symb*kappa1.symb*(GamtildeU.symb[i]-GamtildeDU.symb[i])
for i in range(3):
    RHS_GamtildeU.var[i] += Dsymb(GamtildeU.symb[i],'KO')

In [55]:
fileRHS.write(RHS_GamtildeU.AMReXRHS())

4779

In [56]:
mul = stVar('mul')
mus = stVar('mus')

mul.var = 2/alpha.symb
mus.var = 1/(alpha.symb**(2))

In [57]:
RHS_alpha = stVar('alpha')

RHS_alpha.var += -mul.symb*(alpha.symb**2)*Khat.symb+betaDalpha.symb
RHS_alpha.var += Dsymb(alpha.symb,'KO')

In [58]:
fileRHS.write(mul.AMReXReal())
fileRHS.write(mus.AMReXReal())
fileRHS.write(RHS_alpha.AMReXRHS())

96

In [59]:
eta = stVar('eta')
eta.var = 2.0

RHS_betaU = stVarRank1('betaU')

for i in range(3):
    RHS_betaU.var[i] += mus.symb*(alpha.symb**2)*GamtildeU.symb[i]-eta.symb*betaU.symb[i]+betaDbetaU.symb[i]
    
for i in range(3):
    RHS_betaU.var[i] += Dsymb(betaU.symb[i],'KO')

In [60]:
fileRHS.write(eta.AMReXReal())
fileRHS.write(RHS_betaU.AMReXRHS())

342

In [61]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()